# PHREEQC HFO Model Plotting
This notebook will plot your PHREEQC results along with your data. You will need to upload the analyzed adsorption data and the selected output file from PHREEQC. 


### Start this notebook by specifying the data and model from PHREEQC you would like to plot.
* Replace `df_example.csv` with the filename containing your HFO adsorption data.
* Replace `selected_example.sel` with either PHREEQC model output `HFO_afternoon.sel` or `HFO_morning.sel`.

In [ ]:
# Replace the filename with the file containing your adsorption data
data_set = 'df_example.csv'
# Replace the filename with the appropriate PHREEQC model
model_selected_output = 'selected_example.sel'

In [ ]:
# Import the pandas module to store data in a dataframe
import pandas as pd
import os

# Load the data set
df1 = pd.read_csv(data_set)
# This will automatically remove data that is below pH 4.5 or above pH9.5
df2 = df1[(df1['pH'] >= 4.5) & (df1['pH'] <= 9.5)][['pH', 'sampconc']]
# This will sort the data in ascending order based on pH
df2.sort_values("pH", inplace=True)
# This will calculate Zn adsorption as a percentage of the total Zn concentration
df2['Znadsper'] = (1 - (df2['sampconc'] / df1["tot_zn_ppm"][0])) * 100
# This will replace negative values of Znadsper with 0
df2['Znadsper'] = df2['Znadsper'].clip(lower=0)
# This will replace Znadsper values greater than 100 with 100
df2['Znadsper'] = df2['Znadsper'].clip(upper=100)

# load the selected output into a dataframe and specify that the data are separated by tabs
mod = pd.read_csv(model_selected_output, sep="\t")
# get rid of the first row data (see comment above about why)
mod.drop([0], inplace=True)
mod.drop([1], inplace=True)
# get rid of extraneous whitespaces in the column headers
mod = mod.rename(columns=lambda x: x.strip())
# sort the data in acending order based on pH
mod.sort_values("pH", inplace=True)

# Double check that the selected output looks correct

In [ ]:
mod

# Double check that the data looks correct

In [ ]:
df2

### PHREEQC speciates the Zn adsorption onto the strong and weak site separately. Consequently, we need to sum the weak and strong Zn surface complexes at each pH value in order to calculate total Zn adsorbed.

In [ ]:
# Convert the total Zn used in the experiments from ppm to molality
znm = df1["tot_zn_ppm"][0] / 65380
# sum the Zn adsorbed on the strong and weak surface sites at each pH value
modznads = mod["m_Hfo_wOZn+"] + mod["m_Hfo_sOZn+"]
# calculate total zn adsorbed as a percent
modsper = (modznads / znm) * 100
# calcuate Zn adsorbed on the strong and weak site as percents
znstrong = mod["m_Hfo_sOZn+"] / znm * 100
znweak = mod["m_Hfo_wOZn+"] / znm * 100

### Now we can plot the experimental data and model result on the same plot. We can also add the predicted speciation of the Zn surface species.

In [ ]:
# I'll need to import a some modules for the plotting and downloading the plot.
import matplotlib.pyplot as plt

# to make the plots look nicer we will import the seaborn module
import seaborn as sns

sns.set_theme()

plt.figure()
# I'll plot the data as points and the model as a line
plt.scatter(df2["pH"], df2["Znadsper"], label="data")
plt.plot(mod["pH"], modsper, label="model")
plt.plot(mod["pH"], znstrong, label="strong", linestyle="--")
plt.plot(mod["pH"], znweak, label="weak", linestyle=":")
# Label the x and y axes
plt.xlabel("pH")
plt.ylabel("Zn (% adsorbed)")

# Add a legend to the plot
plt.legend()

# Save and download the plot as pdf and png file
plt.savefig("zn_hfo_mod.pdf")